# **Space X  Falcon 9 First Stage Landing Prediction**


# NoteBook 3: Launch Site Location Analysis with Folium


In the previous exploratory data analysis notebook, visualization of the SpaceX launch dataset was done using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, more interactive visual analytics using `Folium`, will be performed.


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives


Find some geographical patterns about launch sites
-  Mark all launch sites on a map
-  Mark the success/failed launches for each site on the map
-  Calculate the distances between a launch site to its proximities

----

## Import Libraries

In [1]:
import folium   # for visualizing geospatial data
import pandas as pd   # for data manipulation and analysis.


from folium.plugins import MarkerCluster  # for point layers present on the map
from folium.plugins import MousePosition  # for coordinates of mouse position
from folium.features import DivIcon # Draw text on Map

### Launch Site Location Analysis


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [2]:
df = pd.read_csv('data/spacex_launch_geo.csv')
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, take a look at what are the coordinates for each site.


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = df.groupby(['Launch Site'], as_index=False).first() #groupby 'Launch Site' and compute the first non-null entry for each column
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give any intuitive insights about where are those launch sites. Visualize those locations by pinning them on a map.


First create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate.

In [5]:
# Create an Orange circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create an icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, add a circle for each launch site in data frame `launch_sites_df`


In [6]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

for index, row in launch_sites_df.iterrows(): 
    #define the position and name of sites
    lat = row['Lat']
    long = row['Long']
    launch_site = row['Launch Site']
    
    #Define the Circle object
    circle = folium.Circle(
                #properties of circle (location, radius, color, fill)
                [lat, long],
                radius=50, 
                color='#d35400', 
                fill=True
            ).add_child(folium.Popup(launch_site)) #popup label of site name
    
    #Add the Circle objects to the map (in the loop)
    site_map.add_child(circle)
    
site_map

All SpaceX launch sites are on coasts of the United States of America, specifically Florida and California.

### Mark the success/failed launches for each site on the map


Next, try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
The `df` dataframe has detailed launch records, and the `class` column indicates if this launch was successful or not.


In [7]:
df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


Create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters were used to simplify the map containing many markers having the same coordinate.


In [8]:
# Create MarkerCluster object
marker_cluster = MarkerCluster()

# Function to check the value of `class` column
def assign_marker_color(launch_outcome):
    # If class=1, marker_color value will be green
    # If class=0, marker_color value will be red
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
df['marker_color'] = df['class'].apply(assign_marker_color)
df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in `df` data frame, add a `folium.Marker` to `marker_cluster`


In [9]:
# put launches into clusters and put onto map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster) # note, this won't show up unless the individual launches are put onto the map (the next lines of code cover this)


for index, row in df.iterrows():
    
    # define the position and launch site for each launch
    lat = row['Lat']
    long = row['Long']
    launch_site = row['Launch Site']
    
    # define the launch marker for each launch
    launch_marker = folium.Marker(
        [lat, long],
        # an icon as a text label and colour depending on launch success/failure (marker colour)
        icon=folium.Icon(color='white', icon_color=row['marker_color'])
    )
    
    # Add the launch marker for each launch to the launch clusters (not the map, the launch clusters themselves)
    marker_cluster.add_child(launch_marker)

#show the map zoomed out with the centre at NASA
site_map

`Zoom in` to see that the Launches have been grouped into clusters, and annotated with `green icons` for successful launches, and `red icons` for failed launches.

### Calculate the distances between a launch site to its proximities


Need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while exploring the map, coordinates of any points of interests (such as railway) can easily find the 

In [10]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move the mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    '''
    Calculate the distance between 2 points on the map.
        Parameters:
            lat1: Latitude of point 1
            lon1: Logitude of point 1
            lat2: Latitude of point 2
            lon2: Logitude of point 2
        Returns:
            distance: the distance between the 2 points

    '''
    # approximate radius of earth in km
    R = 6373.0

    # Convert from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Get radian difference between the 2 points
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [12]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56327  Lon: -80.57691
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
launch_site_lat = 28.56327
launch_site_long = -80.57691
launch_site_coordinates = [launch_site_lat, launch_site_long]

coastline_lat = launch_site_lat #same latitude, so the line is directly east
coastline_long = -80.56781
coastline_coordinates = [coastline_lat, coastline_long]

distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)

After obtained its coordinate, create a `folium.Marker` to show the distance


In [13]:
# Create and add a folium.Marker on the selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
distance_marker = folium.Marker(
    coastline_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
#add the distance marker to the map and show the map
site_map.add_child(distance_marker)
site_map

Draw a `PolyLine` between a launch site to the selected coastline point


In [14]:
# define the distance line (between the coastline and the launch site)
distance_line=folium.PolyLine(
    locations=[launch_site_coordinates, coastline_coordinates],
    weight=1
)

#add the distance line to the map and show the map
site_map.add_child(distance_line)
site_map

Similarly, draw a line betwee a launch site to its closest highway, railway and city. You need to use `MousePosition` to find the their coordinates on the map first


A Highway map symbol may look like this   
![Highway image](images/highway.png)

A Railway map symbol may look like this   
![Railway image](images/railway.png)

A City map symbol may look like this   
![City image](images/city.png)

In [17]:
# Create a marker with distance to a closest city, railway, highway.
# Draw a line between the marker to the launch site

def distance_and_line(destination_coordinates):
    '''
    Create a marker with distance to a closest city, railway and highway.
    Draw a line between the marker to the launch site
    
    '''
    
    #calculate the distance between the chosen destination and the launch site defined previously
    distance_to_destination = calculate_distance(launch_site_lat, launch_site_long, destination_coordinates[0], destination_coordinates[1])
    
    
    #define the distance marker (which will be shown at the chosen destination)
    distance_marker = folium.Marker(
        destination_coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#ff5c33;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_destination),
            )
        )

    #define the distance line (between the chosen destination and the launch site)
    distance_line=folium.PolyLine(
        locations=[launch_site_coordinates, destination_coordinates],
        weight=1
    )

    #add the distance marker to the map 
    site_map.add_child(distance_marker)
    #add the distance line to the map 
    site_map.add_child(distance_line)

    
closest_city_coordinates = [28.10106, -80.6369]
closest_railway_coordinates = [28.57208, -80.58528]
closest_highway_coordinates = [28.56327, -80.57076]

#add distance markers and lines for closest city, railway and highway, using the above function
distance_and_line(closest_city_coordinates)
distance_and_line(closest_railway_coordinates)
distance_and_line(closest_highway_coordinates)

#show the map
site_map

Placement of launch sites can be understood more after plotting distance lines to the proximities.

- Are launch sites in close proximity to railways?
    
    `YES`. The nearest railway is only `1.28 km` away.

- Are launch sites in close proximity to highways?
    
    `YES`. The nearest highway is only `0.60 km` away. 

- Are launch sites in close proximity to coastline?
    
    `YES`. The coastline is only `0.89 km` away.

- Do launch sites keep certain distance away from cities?
    
    `YES`. The nearest city is `51.75 km` away


--End of Notebook--

# Next Steps:

In this notebook many interesting insights related to the launch sites' location were discovered using folium, in a very interactive way. Next, building a `dashboard` using `Ploty Dash` on detailed launch records.


----